In [ ]:
# default_exp models

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# Models

>Description

In [ ]:
#export
#imports
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

In [ ]:
#export
class _FaceClassificationCNN(nn.Module):
    
    def __init__(self,
                 n_ch_input,
                 n_classes):
        super(_FaceClassificationCNN, self).__init__()

        self.n_ch_input = n_ch_input
        self.n_classes = n_classes

        self.conv_layer_1 = nn.Conv2d(in_channels=n_ch_input,
                                      out_channels=128,
                                      kernel_size=7)
        self.relu_layer_1 = nn.ReLU()
        self.pool_layer_1 = nn.MaxPool2d(kernel_size=2,
                                         stride=2)

        self.conv_layer_2 = nn.Conv2d(in_channels=128,
                                      out_channels=64,
                                      kernel_size=7)
        self.relu_layer_2 = nn.ReLU()
        self.pool_layer_2 = nn.MaxPool2d(kernel_size=2,
                                         stride=2)

        self.relu_layer_3 = nn.ReLU()
        self.linear_layer_3 = nn.Linear(in_features=64*11*11,
                                        out_features=n_classes)
        
        self.relu_layer_4 = nn.ReLU()
        self.linear_layer_4 = nn.Linear(in_features=n_classes,
                                        out_features=n_classes)

    def forward(self, x):
        x = self.conv_layer_1(x)
        x = self.relu_layer_1(x)
        x = self.pool_layer_1(x)
        x = self.conv_layer_2(x)
        x = self.relu_layer_2(x)
        x = self.pool_layer_2(x)
        x = x.view(-1, 64*11*11)
        x = self.relu_layer_3(x)
        x = self.linear_layer_3(x)
        x = self.relu_layer_4(x)
        x = self.linear_layer_4(x)
        return x
        


In [ ]:
#export
class FaceClassificationCNN():
    def __init__(self,
                 n_ch_input,
                 n_classes,
                 lr,
                 lr_decay,
                 n_lr_decay_steps,
                 n_epochs,
                 eval_steps):

        # Architecture parameters
        self.n_ch_input = n_ch_input
        self.n_classes = n_classes

        # Optimization parameters
        self.lr = lr
        self.lr_decay = lr_decay
        self.n_lr_decay_steps = n_lr_decay_steps
        self.n_epochs = n_epochs
        self.eval_steps = eval_steps

        self.model = _FaceClassificationCNN(n_ch_input=n_ch_input,
                                            n_classes=n_classes)

        # self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def fit(self, train_loader, val_loader):
        print("="*30 + 'Start Fitting' + "="*30)
        # self.model.to(self.device)
        self.model.train()
        loss_function = nn.CrossEntropyLoss()
        optimizer = Adam(self.model.parameters(), 
                         lr=self.lr, 
                         weight_decay=0.0005)
        scheduler = StepLR(optimizer=optimizer, 
                           step_size=self.n_epochs//self.n_lr_decay_steps,
                           gamma=self.lr_decay)
        
        self.train_loss = -1
        self.val_loss = -1
        self.trajectories = {'epoch': [],
                             'train_loss': [],
                             'val_loss': []}

        for epoch in range(self.n_epochs):
            
            train_loss = 0

            for batch_idx, (img, label) in enumerate(train_loader):
                
                # img = img.to(self.device)
                # label = label.to(self.device)
                
                output = self.model(img)

                loss = loss_function(output, label)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                scheduler.step()

                train_loss += loss.item()
            
            train_loss /= len(train_loader)         

            if epoch % self.eval_steps == 0:
                val_loss, val_accuracy = self.evaluate_performance(val_loader)
        
                self.trajectories['epoch'].append(epoch)
                self.trajectories['train_loss'].append(train_loss)
                self.trajectories['val_loss'].append(val_loss)

                display_str = f'epoch: {epoch} '
                display_str += f'train_loss: {np.round(train_loss,4)} '
                display_str += f'val_loss: {np.round(val_loss,4)} '
                display_str += f'val_accuracy: {np.round(val_accuracy,4):.2%}'
                print(display_str)

                if self.val_loss > val_loss: self.val_loss = val_loss
                if self.train_loss > train_loss: self.train_loss = train_loss
        
        print("="*72+"\n")


    def evaluate_performance(self, val_loader):

        loss_function = nn.CrossEntropyLoss()
        # self.model.to(device)
        self.model.eval()

        val_loss = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0
        
        with torch.no_grad():
            for batch_idx, (img, label) in enumerate(val_loader):
                
                # img = img.to(device)
                # label = label.to(device)
                outputs = self.model(img)
                loss = loss_function(outputs, label).detach()
                val_loss += loss.item()

                predicted = torch.argmax(outputs, 1)
                total_predictions += img.size(0)
                correct_predictions += (predicted == label).sum().item()

        val_loss /= len(val_loader) 
        acc = correct_predictions/total_predictions
        
        return val_loss, acc


In [ ]:
from axa_hw2p2.datasets import FaceClassificationDataset
import os
import numpy as np
import torch
from torch.utils.data import Dataset
import torchvision
from PIL import Image
import pandas as pd
from torch.utils.data import DataLoader


In [ ]:
np.random.seed(1)
sample = np.array(range(20))
train_dataset = FaceClassificationDataset(sample, mode='train')
val_dataset = FaceClassificationDataset(sample, mode='val')

In [ ]:
# Architecture parameters
mc = {}
mc['n_ch_input'] = 3
mc['n_classes'] = len(np.unique([t[1] for t in train_dataset]))

# Optimization and regularization parameters
mc['batch_size'] = 1
mc['lr'] = 0.001
mc['lr_decay'] = 1
mc['n_lr_decay_steps'] = 2
mc['n_epochs'] = 16
mc['eval_steps'] = 4

print(77*'=')
print(pd.Series(mc))
print(77*'=')

In [ ]:
model = FaceClassificationCNN(n_ch_input=mc['n_ch_input'],
                              n_classes=mc['n_classes'],
                              lr=mc['lr'],
                              lr_decay=float(mc['lr_decay']),
                              n_lr_decay_steps=int(mc['n_lr_decay_steps']),
                              n_epochs=mc['n_epochs'],
                              eval_steps=mc['eval_steps'])
print(model.model)

In [ ]:
train_loader = DataLoader(train_dataset, 
                          shuffle=True, 
                          batch_size=mc['batch_size'], 
                          drop_last=True)
val_loader = DataLoader(val_dataset, 
                        shuffle=False, 
                        batch_size=mc['batch_size'], 
                        drop_last=True)



In [ ]:
model.fit(train_loader=train_loader, val_loader=val_loader)